## AutoML Prediction Process
###  1) Read data
###  2) Load Model
###  3) RUn predictions

In [1]:
# import sklearn.external.joblib as extjoblib
import joblib

from copy import copy

import numpy as np
from numpy import inf
import pandas as pd
from datetime import datetime
import functools
import matplotlib.pyplot as plt  

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import svm 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier  #RF and GBM algorithm
from sklearn.linear_model import ElasticNet, SGDClassifier
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn import preprocessing, neighbors, metrics
import sklearn
if sklearn.__version__<'0.20':
    from sklearn.cross_validation import train_test_split, KFold, StratifiedKFold, PredefinedSplit
else:
    from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, PredefinedSplit

import scipy.stats as st
from sklearn.metrics import mean_absolute_error, accuracy_score, log_loss, make_scorer, auc, roc_auc_score


%matplotlib inline

/tmp/ipykernel_347572/1290565168.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import automl
from automl import StackLayer, TextElasticNetBinary, MissingDataHandler

In [3]:
#https://www.hindawi.com/journals/bmri/2014/781670/
input_data=pd.read_csv("diabetes_data.csv")
input_data.sample(5)

,rowID,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted,diag_1_desc,diag_2_desc,diag_3_desc
3502,3503,Caucasian,Male,[60-70),?,Urgent,Discharged to home,Physician Referral,12,MC,...,No,No,No,No,No,No,False,Secondary malignant neoplasm of lung,"Urinary tract infection, site not specified",Secondary malignant neoplasm of lung
5377,5378,Caucasian,Female,[70-80),?,Elective,Discharged to home,Physician Referral,1,HM,...,No,No,No,No,Ch,Yes,False,Renal colic,"Chronic airway obstruction, not elsewhere clas...",Unspecified prolapse of vaginal walls
58,59,Caucasian,Female,[60-70),?,Emergency,Discharged/transferred to home with home healt...,NaN,3,?,...,No,No,No,No,Ch,Yes,True,Diabetes mellitus without mention of complicat...,Hyperosmolality and/or hypernatremia,Paroxysmal supraventricular tachycardia
65,66,Caucasian,Female,[50-60),?,NaN,Discharged to home,NaN,2,?,...,No,No,No,No,Ch,Yes,False,"Respiratory abnormality, unspecified","Respiratory abnormality, unspecified",Diabetes mellitus without mention of complicat...
9868,9869,AfricanAmerican,Male,[60-70),?,Emergency,Discharged to home,Emergency Room,3,DM,...,No,No,No,No,Ch,Yes,True,Malignant hypertensive heart disease without h...,Diabetes mellitus without mention of complicat...,Paroxysmal supraventricular tachycardia


In [4]:
target='readmitted'
model_file='pipeline.pkl'

In [5]:
X=input_data
y=input_data[target].astype(int)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Run predictions

In [7]:
best_model = joblib.load(model_file)

In [8]:
test_pred2=best_model.predict_proba(X_test[list(best_model.feature_names_in_)])
roc_auc_score(y_test,test_pred2[:,1])

0.6983620102322787